In [9]:
import tensorflow as tf
import cv2
import numpy as np

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing
from data.dataloader import DataLoader

In [10]:
def idx2class():
    return dict((v, k) for k, v in Config.pascal_voc_classes.items())


def draw_boxes_on_image(image, boxes, scores, classes):
    num_boxes = boxes.shape[0]
    for i in range(num_boxes):
        class_and_score = str(idx2class()[classes[i]]) + ": " + str(scores[i])
        cv2.rectangle(img=image, pt1=(boxes[i, 0], boxes[i, 1]), pt2=(boxes[i, 2], boxes[i, 3]), color=(255, 0, 0), thickness=2)
        cv2.putText(img=image, text=class_and_score, org=(boxes[i, 0], boxes[i, 1] - 10), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1.5, color=(0, 255, 255), thickness=2)
    return image

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    interpreter.allocate_tensors()
    return interpreter

def test_single_picture(picture_dir, model):
    image_array = cv2.imread(picture_dir)
    image = DataLoader.image_preprocess(is_training=False, image_dir=picture_dir)
    image = tf.expand_dims(input=image, axis=0)
    
    input_data = np.array(image, dtype = np.uint8)
    input_details = model.get_input_details()[0]
    output_details = model.get_output_details()[0]
    model.set_tensor(input_details['index'], input_data)
    model.invoke()
    outputs= model.get_tensor(output_details['index'])
    print(outputs)
    # 원래 outputs =  소수점 형태
    # quantizaation model =  숫자 너무큼.. output_tensor shape 해결하기
    
    
    
    outputs = tf.constant(outputs)
    
    post_process = PostProcessing()
    boxes, scores, classes = post_process.testing_procedure(outputs, [image_array.shape[0], image_array.shape[1]])
    print("-"*50)
    print("boxes")
    print(boxes)
    print("-" * 50)
    print("scores")
    print(scores)
    print("-" * 50)
    print("classes")
    print(classes)
    print("-" * 50)
    image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
    return image_with_boxes

test_image_dir = "./test_pictures/ck0kfhu4n8q7f0701ixmonyig_jpeg.rf.a3cc5282520b3bac90718bdd5528bd76.jpg"

In [13]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

model_path = "C:/Users/yutan/Desktop/EfficientDet_TensorFlow2/tflite_model_quant.tflite"
efficientdet_lite = load_model(model_path)

image = test_single_picture(picture_dir=test_image_dir, model=efficientdet_lite)

cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
cv2.imshow("detect result", image)
cv2.waitKey(0)

[[[ 0.00297724 -0.03572687  0.         -0.00297724  0.49124447]
  [-0.01786344 -0.0833627   0.02084067 -0.00893172  0.5091079 ]
  [ 0.         -0.04168135 -0.07443098  0.01786344  0.48826724]
  ...
  [-0.00595448  0.03572687  0.05656755  0.03572687  0.49124447]
  [-0.03274963  0.04168135 -0.05061307  0.00595448  0.5388803 ]
  [-0.04763583 -0.04763583 -0.00595448  0.00297724  0.5001762 ]]]
--------------------------------------------------
boxes
[[ 17.467577   0.        52.744617  45.939415]
 [ 37.45707    0.        72.75512   45.939415]
 [ 57.45707    0.        92.75513   45.939415]
 [ 77.45707    0.       112.75513   45.939415]
 [ 97.45706    0.       132.75513   45.939415]
 [117.45706    0.       152.75513   45.939415]
 [137.45706    0.       172.75513   45.939415]
 [157.45709    0.       192.75513   45.939415]
 [177.45709    0.       212.75513   45.939415]
 [197.45709    0.       232.75513   45.939415]
 [217.45709    0.       252.75513   45.939415]
 [237.45709    0.       272.75513 

-1

In [14]:
# 원래 모델
def test_single_picture(picture_dir, model):
    image_array = cv2.imread(picture_dir)
    image = DataLoader.image_preprocess(is_training=False, image_dir=picture_dir)
    image = tf.expand_dims(input=image, axis=0)

    outputs = model(image, training=False)
    print(outputs)
    post_process = PostProcessing()
    boxes, scores, classes = post_process.testing_procedure(outputs, [image_array.shape[0], image_array.shape[1]])
    print("-"*50)
    print("boxes")
    print(boxes)
    print("-" * 50)
    print("scores")
    print(scores)
    print("-" * 50)
    print("classes")
    print(classes)
    print("-" * 50)
    image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
    return image_with_boxes

if __name__ == '__main__':
    # GPU settings
    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    efficientdet = EfficientDet()
    efficientdet.load_weights(filepath=Config.save_model_dir + "saved_model")

    image = test_single_picture(picture_dir=test_image_dir, model=efficientdet)

    cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
    cv2.imshow("detect result", image)
    cv2.waitKey(0)

tf.Tensor(
[[[ 3.3581583e+00 -1.8893352e-01 -8.0869716e-01 -1.5753392e+00
    1.5497410e-04]
  [-4.0502977e+00 -2.1152582e+00 -2.6471562e+00 -1.4114972e+00
    2.0257104e-02]
  [-9.5528231e+00 -2.3242028e+00  2.6107430e+00  1.5415084e+00
    1.9061383e-02]
  ...
  [-7.8000035e+00  8.3751793e+00 -2.2799952e+01 -2.6449896e+01
    0.0000000e+00]
  [-1.4209638e+01  3.1146093e+01  4.3055035e+01 -2.7493441e+00
    0.0000000e+00]
  [ 1.7430079e+01 -1.2523052e+01 -1.7156477e+01 -5.5715279e+01
    2.5651483e-23]]], shape=(1, 49104, 5), dtype=float32)
--------------------------------------------------
boxes
[[  0.       643.0637    45.794323 680.66797 ]]
--------------------------------------------------
scores
[0.4328637]
--------------------------------------------------
classes
[0]
--------------------------------------------------
